## Hardware check

In [1]:
# gpu check
! nvidia-smi

Mon Jun  5 12:48:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   36C    P0    40W / 300W |      9MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [1]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
pwd

'/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis'

In [4]:
import pickle
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.config.model_args import ModelArgs
import resources.smart_cond as sc

2023-06-05 11:33:40.680507: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-05 11:33:41.263842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 11:33:54.915253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Version Check

In [6]:
! python --version

Python 3.9.7


In [7]:
import tensorflow as tf
print(tf. __version__)

2.12.0


In [8]:
pickle.format_version

'4.0'

## Load Data

In [9]:
# ! unzip Data/sepsis_removed_0.pkl.zip

In [10]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
del pkl

text_data = data[data['variable'] == 'Text']
text_data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.0,1.0
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.0,1.0
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.0,1.0
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.0,1.0
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.0,1.0
...,...,...,...,...,...,...,...
1407425,31015,99.400000,Text,NPN\n\n\n#1 Infant remains in RA with O2 sats...,noteevents,1.0,1.0
1407426,31015,105.783333,Text,"Neonatology\nDOL #5, CGA 36 weeks. \n\nCVR: Co...",noteevents,1.0,1.0
1407427,31015,112.650000,Text,Family Meeting Note\nFamily meeting held with ...,noteevents,1.0,1.0
1407428,31015,114.033333,Text,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...,noteevents,1.0,1.0


In [12]:
texts = text_data['value']
texts

0          Admission Date:  [**2119-5-4**]              D...
1          Admission Date:  [**2112-12-8**]              ...
2          Admission Date:  [**2194-7-18**]              ...
3          Admission Date:  [**2194-1-7**]              D...
4          Admission Date:  [**2186-6-7**]     Discharge ...
                                 ...                        
1407425    NPN\n\n\n#1  Infant remains in RA with O2 sats...
1407426    Neonatology\nDOL #5, CGA 36 weeks. \n\nCVR: Co...
1407427    Family Meeting Note\nFamily meeting held with ...
1407428    NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...
1407429    NPN 1800\nNursing Addendum:\n[**Known lastname...
Name: value, Length: 1407430, dtype: object

In [13]:
batch = texts[500000:]
batch

500000     [**2182-3-5**] 11:04 PM\n ABDOMEN (SUPINE ONLY...
500001     [**2161-5-4**] 10:36 AM\n CT HEAD W/O CONTRAST...
500002     [**2123-3-4**] 3:41 PM\n CHEST (PORTABLE AP); ...
500003     [**2180-2-11**] 11:03 AM\n CHEST (PORTABLE AP)...
500004     [**2189-4-17**] 1:37 PM\n CHEST (PORTABLE AP);...
                                 ...                        
1407425    NPN\n\n\n#1  Infant remains in RA with O2 sats...
1407426    Neonatology\nDOL #5, CGA 36 weeks. \n\nCVR: Co...
1407427    Family Meeting Note\nFamily meeting held with ...
1407428    NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...
1407429    NPN 1800\nNursing Addendum:\n[**Known lastname...
Name: value, Length: 907430, dtype: object

In [14]:
model_args = ModelArgs(max_seq_length=512, silent = True)
model = RepresentationModel(
    "bert", "emilyalsentzer/Bio_ClinicalBERT", args=model_args)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# time the process
import time
tic = time.perf_counter()

In [ ]:
# combine_strategy="mean"
features = model.encode_sentences(batch, combine_strategy="mean")

In [ ]:
# batch 1, 4 gpus
toc = time.perf_counter()
print(f"Time consumed generating embeddings for {toc - tic:0.4f} seconds")

### do two batches due to runtime limitation on JupyterLab (max. 4h connection)

In [ ]:
# batch 0
# toc = time.perf_counter()
# print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

## approx. 122 min -> 2 h per batch

In [ ]:
pickle.dump([features], open('features_batch_1.pkl','wb'))

### Add embeddings to `sepsis_removed_0.pkl`

In [3]:
import pickle
import numpy as np

In [4]:
# read data
# del data
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))

In [5]:
# read embeddings
batch_0 = pickle.load(open('tmp/features_batch_0.pkl', 'rb'))
batch_1 = pickle.load(open('tmp/features_batch_1.pkl', 'rb'))

emb_0 = batch_0[0]
emb_1 = batch_1[0]

In [6]:
emb = np.concatenate([emb_0, emb_1])

In [7]:
pkl.append(emb)

In [8]:
pickle.dump(pkl, open('Data/sepsis_removed_0.pkl','wb'))

In [13]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [15]:
pkl[-1].shape

(1407430, 768)